In [7]:
import pandas as pd

In [8]:
test_flight_profile = pd.read_csv("flight-profiles/test-flight-profile.csv")
test_flight_profile

,Phase,Duration (min),Altitude (ft),Speed (kts),Thrust (%)
0,taxi,5.0,0,0,7
1,takeoff,0.5,1500,200,100
2,climb,15.0,35000,280,85
3,cruise,60.0,35000,470,60
4,descent,20.0,0,250,30


In [9]:
engine_data = pd.read_csv("data/engine-data.csv")
engine_data

,Engine Family,Engine Model,Combustor,Fuel,Bypass Ratio,Pressure Ratio,Rated Thrust (kN),Phase,Power Setting (%),Fuel Flow (kg/s),Emission Index HC (g / kg fuel),Emission Index CO (g / kg fuel),Emission Index NOx (g / kg fuel)
0,CFM56,5B4,DAC-II,Jet A,5.9,27.7,120.1,TAKE-OFF,100,1.14,0.1,1.4,18.4
1,CFM56,5B4,DAC-II,Jet A,5.9,27.7,120.1,CLIMB OUT,85,0.95,0.2,3.6,13.6
2,CFM56,5B4,DAC-II,Jet A,5.9,27.7,120.1,APPROACH,30,0.34,5.3,29.1,6.5
3,CFM56,5B4,DAC-II,Jet A,5.9,27.7,120.1,IDLE,7,0.12,3.6,40.1,3.9


In [13]:
profile_duration_min = test_flight_profile['Duration (min)']
profile_duration_seconds = profile_duration_min * 60
print(profile_duration_seconds)

0     300.0
1      30.0
2     900.0
3    3600.0
4    1200.0
Name: Duration (min), dtype: float64


In [14]:
import pandas as pd 
import json
import os

# ---Reading Data Files---
test_flight_profile = pd.read_csv("flight-profiles/test-flight-profile.csv")
with open ("data/engines-data.json") as f:
    data = json.load(f)

# ---Extract the phase-wise emission for the CFM56-5B4/2P engine from the JSON datafile---
phases = data["CFM56-5B4/2P"]["Phases"]

# ---Importing Variables from Data---
phase_data = []
for phase_name, details in phases.items():
    ei  = details["Emission Indices (g/kg fuel)"]
    phase_data.append({
        "Phase": phase_name,
        "Fuel Flow (kg/s)": details["Fuel Flow (kg/s)"],
        "HC (g/kg)": ei["HC"],
        "CO (g/kg)": ei["CO"],
        "NOx (g/kg)": ei["NOx"]
    })

phases_df = pd.DataFrame(phase_data)     # Dataframe containing data for all phases

fuel_flow_rates = phases_df["Fuel Flow (kg/s)"] # Extract the fuel flow rates for All Phases (in kg per second)
ei_HC = phases_df["HC (g/kg)"]                  # Extract the Emission Index for Hydrocarbons for All Phases (g/kg of fuel)
ei_CO = phases_df["CO (g/kg)"]                  # Extract the Emission Index for Carbon Monoxide for All Phases (g/kg of fuel)
ei_NOx = phases_df["NOx (g/kg)"]                # Extract the Emission Index for Nitrogen Oxides for All Phases (g/kg of fuel)

phases = test_flight_profile["Phase"] # Extract Phase data from CSV datafile
profile_duration_min = test_flight_profile['Duration (min)'] # Duration for All Phases (minutes)


# ---Calculations--- 
profile_duration_seconds = profile_duration_min * 60  # Duration for All Phases (seconds)

fuel_burned = fuel_flow_rates * profile_duration_seconds # Fuel burned for All Phases (kg)

hc_emissions = ei_HC * fuel_burned      # HC emissions in Fuel Burned
co_emissions = ei_CO * fuel_burned      # CO emissions in Fuel Burned
nox_emissions = ei_NOx * fuel_burned    # NOx emissions in Fuel Burned

total_duration = profile_duration_seconds.sum()     # Total Duration of Flight
total_fuel_burned = fuel_burned.sum()               # Total Fuel Burned
total_co_emissions = co_emissions.sum()             # Total Carbon Monoxide Emissions
total_hc_emissions = hc_emissions.sum()             # Total Hydrocarbon Emissions
total_nox_emissions = nox_emissions.sum()           # Total Nitrogen Oxides Emissions

# ---Create a DataFrame for Total Emissions Summary---
total_data = pd.DataFrame([{
    "Phase":"Total",
    "Duration (s)":total_duration,
    "Fuel Burned (kg)":total_fuel_burned,
    "HC emissions (g)":total_hc_emissions,
    "NOx emissions (g)":total_nox_emissions,
    "CO emissions (g)":total_co_emissions,
}])

# ---Create Phase-wise Emissions Summary DataFrame---
phases_summary_df = pd.DataFrame({
    "Phase":phases,
    "Duration (s)":profile_duration_seconds,
    "Fuel Flow (kg/s)":fuel_flow_rates,
    "Fuel Burned (kg)":fuel_burned,
    "HC emissions (g)":hc_emissions,
    "NOx emissions (g)":nox_emissions,
    "CO emissions (g)":co_emissions,
})
phases_summary_df = pd.concat([phases_summary_df, total_data], ignore_index=True)


# ---Saving Outputs---
output_dir = "output/emissions"
output_file = os.path.join(output_dir, "summary.csv")

phases_summary_df.to_csv(output_file, index=False)
os.makedirs(output_dir, exist_ok=True)


In [15]:
phases_summary_df

,Phase,Duration (s),Fuel Flow (kg/s),Fuel Burned (kg),HC emissions (g),NOx emissions (g),CO emissions (g)
0,taxi,300.0,1.14,342.0,34.2,6292.8,478.8
1,takeoff,30.0,0.95,28.5,5.7,387.6,102.6
2,climb,900.0,0.34,306.0,1621.8,1989.0,8904.6
3,cruise,3600.0,0.12,432.0,1555.2,1684.8,17323.2
4,descent,1200.0,NaN,NaN,NaN,NaN,NaN
5,Total,6030.0,NaN,1108.5,3216.9,10354.2,26809.2


In [ ]:
import plotly.graph_objects as go
import pandas as pd

phases_df = pd.read_csv("output/emissions/summary.csv")

phases = phases_df["Phase"]
co_em = phases_df["CO Emissions (g)"]
nox_em = phases_df["NOx Emissions (g)"]
hc_em = phases_df["HC Emissions (g)"]

fig = go.Figure([
    go.Bar(
           name='HC Emissions', 
           legendgroup='HC',
           x=phases, 
           y=hc_em, 
           text=hc_em.apply(lambda x: f'{x:.3f}'), 
           textposition='inside', 
           textfont=dict(
                size=10,              # Font size of the text on bars
                color='white'         # Font color (e.g., 'white' if inside bar)
            ),
           hovertemplate='Pollutant: HC<br>Phase: %{x}<br>Emissions: %{y:.3f} g<extra></extra>',
           marker=dict(color='#009E73') 
           ),
    go.Bar(
           name='NOx Emissions', 
           legendgroup='NOx',
           x=phases, 
           y=nox_em,
           text=nox_em.apply(lambda x: f'{x:.3f}'),
           textposition='inside',
           textfont=dict(
                size=10,              # Font size of the text on bars
                color='white'         # Font color (e.g., 'white' if inside bar)
            ),
           hovertemplate='Pollutant: NOx<br>Phase: %{x}<br>Emissions: %{y:.3f} g<extra></extra>',
           marker=dict(color='#E69F00') 
           ),
    go.Bar(
           name='CO Emissions', 
           legendgroup='CO',
           x=phases, 
           y=co_em,
           text=co_em.apply(lambda x: f'{x:.3f}'),
           textposition='inside',
           textfont=dict(
                size=10,              # Font size of the text on bars
                color='white'         # Font color (e.g., 'white' if inside bar)
            ),
           hovertemplate='Pollutant: CO<br>Phase: %{x}<br>Emissions: %{y:.3f} g<extra></extra>',
           marker=dict(color='#56B4E9') 
           )
])

fig.update_layout(
    barmode='group',
    yaxis_type='log', 
    template='simple_white',
    title='<b>Engine Emissions by Flight Phase for CFM56-5B4/2P</b><br><sup>Log scale used to highlight variation across pollutants</sup>',

    bargap=0.15,         # Space between groups
    bargroupgap=0.05,    # Space between bars within a group

    # Stylistic Formatting
    height=650,  # Increase vertical space (good for log scale)
    margin=dict(
        l=80,   # Left margin (space for y-axis labels)
        r=40,   # Right margin
        t=100,  # Top margin (space for title + subtitle)
        b=80    # Bottom margin (space for x-axis labels)
    ),
    title_font=dict(
        size=22,
        family="Arial",
        color="black"
    ),
    font=dict(
        size=13, 
        family="Arial"
    ),
    legend=dict(
        font=dict(size=12),
        orientation="h", # horizontal legend
        yanchor="bottom",
        y=-0.2,
        xanchor="center",
        x=0.5
    ),

    # Formatting x-axis
    xaxis=dict(
        title='Flight Phase'
    ),
    
    # Formatting y-axis
    yaxis=dict(
        type='log',
        title='Emissions (g, log scale)',
        tickvals=[1, 10, 100, 1000, 10000, 100000],
        tickformat=',',
        showgrid=True,
        gridcolor='lightgrey',
        gridwidth=1
    )
)

fig.show()
fig.write_html('fig.html')

In [51]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd

# Load and filter data
summary_df = pd.read_csv("output/emissions/summary.csv")
summary_df_excl_total = summary_df[summary_df["Phase"] != "Total"]

# Extract values
phases = summary_df_excl_total["Phase"]
co_em = summary_df_excl_total["CO Emissions (g)"]
nox_em = summary_df_excl_total["NOx Emissions (g)"]
hc_em = summary_df_excl_total["HC Emissions (g)"]


# Sort by value descending
sorted_co_data = sorted(zip(phases, co_em), key=lambda x: x[1], reverse=True)
sorted_nox_data = sorted(zip(phases, nox_em), key=lambda x: x[1], reverse=True)
sorted_hc_data = sorted(zip(phases, hc_em), key=lambda x: x[1], reverse=True)

# Your color palette from largest to smallest
color_palette_co = ["#4197CA", "#64B6E2", "#9AD3F1", "#BCE3F6", "#DEF2FA"]
color_palette_nox = ["#CB8712", "#E69F00", "#F1B733", "#F7CD66", "#FCE399"]
color_palette_hc = ["#00664E", "#009E73", "#33B384", "#66C495", "#99D6B1"]


# Unzip
sorted_co_phases, sorted_co_em = zip(*sorted_co_data)
sorted_nox_phases, sorted_nox_em = zip(*sorted_nox_data)
sorted_hc_phases, sorted_hc_em = zip(*sorted_hc_data)

# Assign colors
sorted_colors_co = color_palette_co[:len(sorted_co_em)]
sorted_colors_nox = color_palette_nox[:len(sorted_co_em)]
sorted_colors_hc = color_palette_hc[:len(sorted_co_em)]

# Create subplot layout
fig = make_subplots(
    rows=1, cols=3,
    column_widths=[0.33, 0.33, 0.33],
    specs=[
        [{"type": "domain"}, {"type": "domain"}, {"type": "domain"}]
    ],
    subplot_titles=(
        "<b>NOₓ Emissions (g)</b><br><sup>Emission of NOx tracked across Phases</sup>", 
        "<b>CO Emissions (g)</b><br><sup>Emission of CO tracked across Phases</sup>", 
        "<b>HC Emissions (g)</b><br><sup>Emission of HC tracked across Phases</sup>"
    )
)

fig.add_trace(
    go.Pie(
        labels=sorted_nox_phases,
        values=sorted_nox_em,
        name="NOx",
        pull=[0.05],
        textinfo='none',
        legendgroup="NOx",
        showlegend=True,
        marker=dict(colors=sorted_colors_nox),
        hovertemplate='Phase: %{label}<br>Percentage: %{percent}<br>Emissions: %{value:.1f} g<extra></extra>',
        domain=dict(y=[0.66, 0.86]),  # Shifted further down
        sort=False,
        direction="clockwise",
        rotation=0
    ),
    row=1, col=1
)

fig.add_trace(
    go.Pie(
        labels=sorted_co_phases,
        values=sorted_co_em,
        name="CO",
        pull=[0.05],
        textinfo='none',
        legendgroup="CO",
        showlegend=True,
        marker=dict(colors=sorted_colors_co),
        hovertemplate='Phase: %{label}<br>Percentage: %{percent}<br>Emissions: %{value:.1f} g<extra></extra>',
        domain=dict(y=[0.36, 0.56]),  # Shifted further down
        sort=False,
        direction="clockwise",
        rotation=0
    ),
    row=1, col=2
)

fig.add_trace(
    go.Pie(
        labels=sorted_hc_phases,
        values=sorted_hc_em,
        name="HC",
        pull=[0.05],
        textinfo='none',
        legendgroup="HC",
        showlegend=True,
        marker=dict(colors=sorted_colors_hc),
        hovertemplate='Phase: %{label}<br>Percentage: %{percent}<br>Emissions: %{value:.1f} g<extra></extra>',
        domain=dict(y=[0.06, 0.26]),  # Shifted further down
        sort=False,
        direction="clockwise",
        rotation=0
    ),
    row=1, col=3
)

# Layout settings
fig.update_layout(
    height=800,
    template='simple_white',
    title_font=dict(size=20),
    legend=dict(
        orientation='h',
        yanchor='bottom',
        y=-0.15, 
        xanchor='center',
        x=0.5,
        title=None,
        font=dict(size=12)
    ),
    margin=dict(l=20, r=20, t=80, b=80),
    font=dict(size=12)
)

fig.show()

In [ ]:
import plotly.graph_objects as go
import pandas as pd

# Load and filter data
summary_df = pd.read_csv("output/emissions/summary.csv")
summary_df_excl_total = summary_df[summary_df["Phase"] != "Total"]
summary_df_total = summary_df[summary_df["Phase"] == "Total"]

# Extract values
phases = summary_df_excl_total["Phase"]
co_em = summary_df_excl_total["CO Emissions (g)"]
nox_em = summary_df_excl_total["NOx Emissions (g)"]
hc_em = summary_df_excl_total["HC Emissions (g)"]

# Sort by value descending
sorted_co_data = sorted(zip(phases, co_em), key=lambda x: x[1], reverse=True)
sorted_nox_data = sorted(zip(phases, nox_em), key=lambda x: x[1], reverse=True)
sorted_hc_data = sorted(zip(phases, hc_em), key=lambda x: x[1], reverse=True)

# Your color palette from largest to smallest
color_palette_co = ["#4197CA", "#64B6E2", "#9AD3F1", "#BCE3F6", "#DEF2FA"]
color_palette_nox = ["#CB8712", "#E69F00", "#F1B733", "#F7CD66", "#FCE399"]
color_palette_hc = ["#00664E", "#009E73", "#33B384", "#66C495", "#99D6B1"]

# Unzip
sorted_co_phases, sorted_co_em = zip(*sorted_co_data)
sorted_nox_phases, sorted_nox_em = zip(*sorted_nox_data)
sorted_hc_phases, sorted_hc_em = zip(*sorted_hc_data)

# Assign colors
sorted_colors_co = color_palette_co[:len(sorted_co_em)]
sorted_colors_nox = color_palette_nox[:len(sorted_co_em)]
sorted_colors_hc = color_palette_hc[:len(sorted_co_em)]

em_data = sorted_nox_em + sorted_co_em + sorted_hc_em
em_phases = (
    [f"NOx / {phase}" for phase in sorted_nox_phases] +
    [f"CO / {phase}" for phase in sorted_co_phases] +
    [f"HC / {phase}" for phase in sorted_hc_phases]
)
em_colors = color_palette_nox + color_palette_co + color_palette_hc


total_co = summary_df_total["CO Emissions (g)"]
total_nox = summary_df_total["NOx Emissions (g)"]
total_hc = summary_df_total["HC Emissions (g)"]

total_em = pd.concat([total_nox, total_co, total_hc])

fig=go.Figure()

# INNER - TOTAL
fig.add_trace(
    go.Pie(
        domain=dict(x=[0.0, 0.5]), 
        labels=["NOx", "CO", "HC"],
        textinfo='none',
        values=total_em,
        hole=0.35,
        sort=False,
        direction='clockwise',
        rotation=0,
        showlegend=False,
        hovertemplate='Pollutant: %{label}<br>Percentage: %{percent}<br>Emissions: %{value:.1f} g<extra></extra>',
        marker=dict(
            colors=("#E69F00","#64B6E2","#009E73"),
            line=dict(
                color='white',
                width=0.5
            )
        )
    )
)

# OUTER - PHASES
fig.add_trace(
    go.Pie(
        domain=dict(x=[0.0, 0.5]),
        labels=em_phases,
        values=em_data,
        textinfo='none',
        hole=0.8,
        sort=False,
        direction='clockwise',
        rotation=0,
        showlegend=True,
        hovertemplate='Pollutant / Phase: %{label}<br>Percentage: %{percent}<br>Emissions: %{value:.1f} g<extra></extra>',
        marker=dict(
            colors=em_colors,
            line=dict(
                color='white',
                width=0.5 
            )
        )
    )
)

fig.update_layout(
    height=800,
    template='simple_white',
    title='<b>Aircraft Engine Emissions Breakdown (in grams)</b><br><sup>Outer Ring shows emission contributions by Flight Phase; Inner Ring summarizes total emissions by Pollutant type</sup>',
    legend_title_text='<b>Phase-wise Breakdown by Pollutant Type</b>',
    legend=dict(
        x=0.75,
        y=0.5,
        xanchor='right',
        yanchor='middle',
        orientation='v',
        font=dict(
            family="Arial",
            size=10,
            color="black"
        )
    ),
    title_font=dict(
        size=22,
        family="Arial",
        color="black"
    )
)

fig.show()

In [103]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd

# Load and filter data
summary_df = pd.read_csv("output/emissions/summary.csv")
summary_df_excl_total = summary_df[summary_df["Phase"] != "Total"]

# Extract values
phases = summary_df_excl_total["Phase"]
co_em = summary_df_excl_total["CO Emissions (g)"]
nox_em = summary_df_excl_total["NOx Emissions (g)"]
hc_em = summary_df_excl_total["HC Emissions (g)"]


# Sort by value descending
sorted_co_data = sorted(zip(phases, co_em), key=lambda x: x[1], reverse=True)
sorted_nox_data = sorted(zip(phases, nox_em), key=lambda x: x[1], reverse=True)
sorted_hc_data = sorted(zip(phases, hc_em), key=lambda x: x[1], reverse=True)

# Your color palette from largest to smallest
color_palette_co = ["#4197CA", "#64B6E2", "#9AD3F1", "#BCE3F6", "#DEF2FA"]
color_palette_nox = ["#CB8712", "#E69F00", "#F1B733", "#F7CD66", "#FCE399"]
color_palette_hc = ["#00664E", "#009E73", "#33B384", "#66C495", "#99D6B1"]

# Unzip
sorted_co_phases, sorted_co_em = zip(*sorted_co_data)
sorted_nox_phases, sorted_nox_em = zip(*sorted_nox_data)
sorted_hc_phases, sorted_hc_em = zip(*sorted_hc_data)

# Assign colors
sorted_colors_co = color_palette_co[:len(sorted_co_em)]
sorted_colors_nox = color_palette_nox[:len(sorted_co_em)]
sorted_colors_hc = color_palette_hc[:len(sorted_co_em)]

sorted_nox_phases = [f"NOx / {phase}" for phase in sorted_nox_phases]
sorted_co_phases = [f"CO / {phase}" for phase in sorted_co_phases]
sorted_hc_phases = [f"HC / {phase}" for phase in sorted_hc_phases]

em_data = sorted_nox_em + sorted_co_em + sorted_hc_em
em_phases = sorted_nox_phases + sorted_co_phases + sorted_hc_phases
em_colors = color_palette_nox + color_palette_co + color_palette_hc

total_co = summary_df_total["CO Emissions (g)"]
total_nox = summary_df_total["NOx Emissions (g)"]
total_hc = summary_df_total["HC Emissions (g)"]

total_em = pd.concat([total_nox, total_co, total_hc])

fig=go.Figure()

# Create subplot layout
fig = make_subplots(
    rows=1, cols=4,
    column_widths=[0.25, 0.25, 0.25, 0.25],
    specs=[
        [{"type": "domain"}, {"type": "domain"}, {"type": "domain"}, {"type": "domain"}]
    ],
    subplot_titles=(
        "<b>NOₓ Emissions (g)</b><br><sup>Emission of NOx tracked across Phases</sup>", 
        "<b>CO Emissions (g)</b><br><sup>Emission of CO tracked across Phases</sup>", 
        "<b>HC Emissions (g)</b><br><sup>Emission of HC tracked across Phases</sup>",
        '<b>Engine Emissions Breakdown (g)</b><br><sup>Outer Ring: Total Data; Inner Ring: Phase-Wise Data</sup>'
    )
)

# INNER - TOTAL
fig.add_trace(
    go.Pie(
        domain=dict(x=[0.0, 0.5]), 
        labels=["NOx", "CO", "HC"],
        textinfo='none',
        values=total_em,
        hole=0.3,
        sort=False,
        direction='clockwise',
        rotation=0,
        showlegend=False,
        hovertemplate='Pollutant: %{label}<br>Percentage: %{percent}<br>Emissions: %{value:.1f} g<extra></extra>',
        marker=dict(
            colors=("#E69F00","#64B6E2","#009E73"),
            line=dict(
                color='white',
                width=0.5
            )
        )
    ),
    row=1, col=4
)

# OUTER - PHASES
fig.add_trace(
    go.Pie(
        domain=dict(x=[0.0, 0.5]),
        labels=em_phases,
        values=em_data,
        textinfo='none',
        hole=0.75,
        sort=False,
        direction='clockwise',
        rotation=0,
        showlegend=False,
        hovertemplate='Pollutant / Phase: %{label}<br>Percentage: %{percent}<br>Emissions: %{value:.1f} g<extra></extra>',
        marker=dict(
            colors=em_colors,
            line=dict(
                color='white',
                width=0.5 
            )
        )
    ),
    row=1, col=4
)

fig.add_trace(
    go.Pie(
        labels=sorted_nox_phases,
        values=sorted_nox_em,
        name="NOx",
        pull=[0.05],
        textinfo='none',
        legendgroup="NOx",
        showlegend=True,
        marker=dict(colors=sorted_colors_nox),
        hovertemplate='Phase: %{label}<br>Percentage: %{percent}<br>Emissions: %{value:.1f} g<extra></extra>',
        domain=dict(y=[0.66, 0.86]),  # Shifted further down
        sort=False,
        direction="clockwise",
        rotation=0
    ),
    row=1, col=1
)

fig.add_trace(
    go.Pie(
        labels=sorted_co_phases,
        values=sorted_co_em,
        name="CO",
        pull=[0.05],
        textinfo='none',
        legendgroup="CO",
        showlegend=True,
        marker=dict(colors=sorted_colors_co),
        hovertemplate='Phase: %{label}<br>Percentage: %{percent}<br>Emissions: %{value:.1f} g<extra></extra>',
        domain=dict(y=[0.36, 0.56]),  # Shifted further down
        sort=False,
        direction="clockwise",
        rotation=0
    ),
    row=1, col=2
)

fig.add_trace(
    go.Pie(
        labels=sorted_hc_phases,
        values=sorted_hc_em,
        name="HC",
        pull=[0.05],
        textinfo='none',
        legendgroup="HC",
        showlegend=True,
        marker=dict(colors=sorted_colors_hc),
        hovertemplate='Phase: %{label}<br>Percentage: %{percent}<br>Emissions: %{value:.1f} g<extra></extra>',
        domain=dict(y=[0.06, 0.26]),  # Shifted further down
        sort=False,
        direction="clockwise",
        rotation=0
    ),
    row=1, col=3
)

# Layout settings
fig.update_layout(
    height=700,
    template='simple_white',
    title_font=dict(size=20),
    legend=dict(
        orientation='h',
        yanchor='bottom',
        y=-0.15, 
        xanchor='center',
        x=0.5,
        title=None,
        font=dict(size=12)
    ),
    margin=dict(l=20, r=20, t=80, b=80),
    font=dict(size=12),
)

fig.show()

In [346]:
import plotly.graph_objects as go 
import pandas as pd

summary_df = pd.read_csv("output/emissions/summary.csv")
summary_df = pd.DataFrame(summary_df)

phases_df = summary_df[summary_df['Phase'] != 'Total'].copy()

phases_df['Start Time (s)'] = phases_df['Duration (s)'].cumsum() - phases_df['Duration (s)']
phases_df['End Time (s)'] = phases_df['Duration (s)'].cumsum()
phases_df['Mid Time (s)'] = (phases_df['Start Time (s)'] + phases_df['End Time (s)']) / 2

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=phases_df['Mid Time (s)'],
    y=phases_df['Fuel Flow (kg/s)'],
    mode='lines+markers',
    fill='tozeroy',
    name='Fuel Used',
    line=dict(color='rgba(255,0,0,0.3)'),
    hoverinfo='skip',
    marker= dict(color="#C44E52"),
    text=phases_df['Phase'],
    hovertemplate= "Fuel Flow: %{y} kg/s<br>Time: %{x} s<extra></extra>"
))

fig.update_layout(
    xaxis_title = 'Time (s)',
    yaxis_title = 'Fuel Flow (kg/s)',
    title='<b>Fuel Flow (kg/s) over Time (s)</b><br><sup>Time series plotted using midpoint' \
    ' timestamps of each phase to represent average fuel flow trends accurately.</sup>',
    template='simple_white',
    showlegend=True,
    xaxis= dict(
        showgrid= True,
        gridwidth= 1,
        gridcolor= 'lightgrey'
    ),
    yaxis= dict(
        showgrid= True,
        gridwidth= 1,
        gridcolor= 'lightgrey'
    )
)

fig.show()

In [347]:
import plotly.graph_objects as go
import pandas as pd

# Load and prepare data
summary_df = pd.read_csv("output/emissions/summary.csv")
phases_df = summary_df[summary_df['Phase'] != 'Total'].copy()

# Compute time points
phases_df['Start Time (s)'] = phases_df['Duration (s)'].cumsum() - phases_df['Duration (s)']
phases_df['End Time (s)'] = phases_df['Duration (s)'].cumsum()
phases_df['Mid Time (s)'] = (phases_df['Start Time (s)'] + phases_df['End Time (s)']) / 2

# Extrapolate first and last slopes
first_mid = phases_df['Mid Time (s)'].iloc[0]
second_mid = phases_df['Mid Time (s)'].iloc[1]
first_flow = phases_df['Fuel Flow (kg/s)'].iloc[0]
second_flow = phases_df['Fuel Flow (kg/s)'].iloc[1]

last_mid = phases_df['Mid Time (s)'].iloc[-1]
second_last_mid = phases_df['Mid Time (s)'].iloc[-2]
last_flow = phases_df['Fuel Flow (kg/s)'].iloc[-1]
second_last_flow = phases_df['Fuel Flow (kg/s)'].iloc[-2]

start_time = phases_df['Start Time (s)'].iloc[0]
end_time = phases_df['End Time (s)'].iloc[-1]

# Calculate slopes and extrapolate
slope_start = (second_flow - first_flow) / (second_mid - first_mid)
extrapolated_start_flow = first_flow - slope_start * (first_mid - start_time)

slope_end = (last_flow - second_last_flow) / (last_mid - second_last_mid)
extrapolated_end_flow = last_flow + slope_end * (end_time - last_mid)

# Prepare extended x and y
x_extended = [start_time] + phases_df['Mid Time (s)'].tolist() + [end_time]
y_extended = [extrapolated_start_flow] + phases_df['Fuel Flow (kg/s)'].tolist() + [extrapolated_end_flow]

# Create plot
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=x_extended,
    y=y_extended,
    mode='lines+markers',
    fill='tozeroy',
    name='Fuel Used',
    line=dict(color='rgba(255,0,0,0.3)'),
    marker=dict(color="#C44E52"),
    text=["Extrapolated Start"] + phases_df['Phase'].tolist() + ["Extrapolated End"],
    hovertemplate="Fuel Flow: %{y:.2f} kg/s<br>Time: %{x:.0f} s<extra></extra>"
))

fig.update_layout(
    xaxis_title='Time (s)',
    yaxis_title='Fuel Flow (kg/s)',
    title='<b>Fuel Flow (kg/s) over Time (s)</b><br><sup>Midpoints used for phase-level fuel flow; slope extended to span full mission time</sup>',
    template='simple_white',
    showlegend=True,
    xaxis=dict(showgrid=True, gridwidth=1, gridcolor='lightgrey'),
    yaxis=dict(showgrid=True, gridwidth=1, gridcolor='lightgrey')
)

fig.show()


In [367]:
import plotly.graph_objects as go
import pandas as pd

# Load your emission summary CSV
summary_df = pd.read_csv("output/emissions/summary.csv")
phases_df = summary_df[summary_df['Phase'] != 'Total'].copy()

# Compute key time points
phases_df['Start Time (s)'] = phases_df['Duration (s)'].cumsum() - phases_df['Duration (s)']
phases_df['End Time (s)'] = phases_df['Duration (s)'].cumsum()
phases_df['Mid Time (s)'] = (phases_df['Start Time (s)'] + phases_df['End Time (s)']) / 2

# Get values for flat extrapolation
start_time = phases_df['Start Time (s)'].iloc[0]
end_time = phases_df['End Time (s)'].iloc[-1]
first_flow = phases_df['Fuel Flow (kg/s)'].iloc[0]
last_flow = phases_df['Fuel Flow (kg/s)'].iloc[-1]

# Construct extended x and y
x_extended = [start_time] + phases_df['Mid Time (s)'].tolist() + [end_time]
y_extended = [first_flow] + phases_df['Fuel Flow (kg/s)'].tolist() + [last_flow]

# Plot with filled area
fig = go.Figure()


fig.add_trace(go.Scatter(
    x=x_extended,
    y=y_extended,
    mode='lines+markers',
    name='Fuel Flow (kg/s)',
    line=dict(color='rgba(255,0,0,0.3)'),
    marker=dict(color="#C44E52"),
    text=["Flat Start"] + phases_df['Phase'].tolist() + ["Flat End"],
    hovertemplate="Fuel Flow: %{y:.2f} kg/s<br>Time: %{x:.0f} s<extra></extra>"
))

fig.add_trace(go.Scatter(
    x=x_extended,
    y=y_extended,
    mode='lines',
    fill='tozeroy',
    name='Fuel Used (kg)',
    line=dict(color='rgba(255,0,0,0.3)'),
    marker=dict(color="#C44E52"),
    text=["Flat Start"] + phases_df['Phase'].tolist() + ["Flat End"],
    hovertemplate="Fuel Flow: %{y:.2f} kg/s<br>Phase: %{text} s<extra></extra>"
))

fig.update_layout(
    xaxis_title='Time (s)',
    yaxis_title='Fuel Flow (kg/s)',
    title='<b>Fuel Flow (kg/s) over Time (s)</b><br><sup>Midpoints used for phase-level fuel flow; extended flat to full mission time</sup>',
    template='simple_white',
    showlegend=True,
    xaxis=dict(
        showgrid=True,
        gridwidth=1,
        gridcolor='lightgrey',
        range=[0 , x_extended[-1]]
    ),
    yaxis=dict(
        showgrid=True, 
        gridwidth=1, 
        gridcolor='lightgrey'
    )
)

fig.show()


In [395]:
import plotly.graph_objects as go
import pandas as pd

# Load your emission summary CSV
summary_df = pd.read_csv("output/emissions/summary.csv")
phases_df = summary_df[summary_df['Phase'] != 'Total'].copy()

# Extract values
phases = summary_df_excl_total["Phase"]
co_em = summary_df_excl_total["CO Emissions (g)"]
nox_em = summary_df_excl_total["NOx Emissions (g)"]
hc_em = summary_df_excl_total["HC Emissions (g)"]

# Compute key time points
phases_df['Start Time (s)'] = phases_df['Duration (s)'].cumsum() - phases_df['Duration (s)']
phases_df['End Time (s)'] = phases_df['Duration (s)'].cumsum()
phases_df['Mid Time (s)'] = (phases_df['Start Time (s)'] + phases_df['End Time (s)']) / 2

# Get values for flat extrapolation
start_time = phases_df['Start Time (s)'].iloc[0]
end_time = phases_df['End Time (s)'].iloc[-1]
nox_start = phases_df['NOx Emissions (g)'].iloc[0]
nox_end = phases_df['NOx Emissions (g)'].iloc[-1]
co_start = phases_df['CO Emissions (g)'].iloc[0]
co_end = phases_df['CO Emissions (g)'].iloc[-1]
hc_start = phases_df['HC Emissions (g)'].iloc[0]
hc_end = phases_df['HC Emissions (g)'].iloc[-1]

# Construct extended x and y
time_extended = [start_time] + phases_df['Mid Time (s)'].tolist() + [end_time]
nox_extended = [nox_start] + phases_df['NOx Emissions (g)'].tolist() + [nox_end]
co_extended = [co_start] + phases_df['CO Emissions (g)'].tolist() + [co_end]
hc_extended = [hc_start] + phases_df['HC Emissions (g)'].tolist() + [hc_end]

# Plot with filled area
fig = go.Figure()


fig.add_trace(go.Scatter(
    x=time_extended,
    y=nox_extended,
    mode='lines+markers',
    name='NOx Emitted (g)',
    line=dict(color='#E69F00'),
    marker=dict(color="#E69F00"),
    text=["Flat Start"] + phases.tolist() + ["Flat End"],
    hovertemplate="NOx Emitted: %{y:.2f} g<br>Time: %{x:.0f} s<br>Phase: %{text}<extra></extra>"
))

fig.add_trace(go.Scatter(
    x=time_extended,
    y=co_extended,
    mode='lines+markers',
    name='CO Emitted (g)',
    line=dict(color='#56B4E9'),
    marker=dict(color="#56B4E9"),
    text=["Flat Start"] + phases.tolist() + ["Flat End"],
    hovertemplate="NOx Emitted: %{y:.2f} g<br>Time: %{x:.0f} s<br>Phase: %{text}<extra></extra>"
))

fig.add_trace(go.Scatter(
    x=time_extended,
    y=hc_extended,
    mode='lines+markers',
    name='HC Emitted (g)',
    line=dict(color='#009E73'),
    marker=dict(color="#009E73"),
    text=["Flat Start"] + phases.tolist() + ["Flat End"],
    hovertemplate="NOx Emitted: %{y:.2f} g<br>Time: %{x:.0f} s<br>Phase: %{text}<extra></extra>"
))

fig.update_layout(
    xaxis_title='Time (s)',
    yaxis_title='Fuel Flow (kg/s)',
    title='<b>Emissions (g) over Time (s)</b><br><sup>Midpoints used for phase-level Emissions; extended flat to full mission time</sup>',
    template='simple_white',
    showlegend=True,
    xaxis=dict(
        showgrid=True,
        gridwidth=1,
        gridcolor='lightgrey',
        range=[0 , x_extended[-1]]
    ),
    yaxis=dict(
        type='log',
        title='Emissions (g)',
        showgrid=True,
        gridcolor='lightgrey',
        gridwidth=1,
        minor=dict(ticklen=1, showgrid=True)
    )
)

fig.show()



In [2]:
# TESTING PLOT-EMISSIONS AFTER STEP 5

import pandas as pd

from plot_emissions import plot_bar_summary
from plot_emissions import plot_pie_summary
from plot_emissions import plot_fuel_flow_summary
from plot_emissions import plot_emissions_line_summary

summary_df = pd.read_csv("output/emissions/summary.csv")

In [3]:
summary_df

,Phase,Duration (s),Fuel Flow (kg/s),Fuel Burned (kg),HC Emissions (g),NOx Emissions (g),CO Emissions (g)
0,Idle/Taxi,300.0,0.12,36.0,129.60,140.40,1443.60
1,Take-Off,30.0,1.14,34.2,3.42,629.28,47.88
2,Climb Out,900.0,0.65,585.0,117.00,7897.50,585.00
3,Cruise,3600.0,0.95,3420.0,684.00,46512.00,12312.00
4,Descent,1200.0,0.34,408.0,2162.40,2652.00,11872.80
5,Total,6030.0,NaN,4483.2,3096.42,57831.18,26261.28


In [4]:
plot_bar_summary(summary_df)
plot_pie_summary(summary_df)
plot_fuel_flow_summary(summary_df)
plot_emissions_line_summary(summary_df)

In [5]:
import sqlite3
import pandas as pd

# Connect to the SQLite database
conn = sqlite3.connect("global_airports_sqlite.db")

# List available tables
tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn)
print(tables)

# Read a specific table (replace 'airports' with actual table name after checking)
df = pd.read_sql_query("SELECT * FROM airports;", conn)

# Preview the data
print(df.head())

# Close connection (optional, but recommended)
conn.close()


              name
0         airports
1  sqlite_sequence
   id icao_code iata_code         name         city           country  \
0   1      AYGA       GKA       GOROKA       GOROKA  PAPUA NEW GUINEA   
1   2      AYLA       LAE          N/A          LAE  PAPUA NEW GUINEA   
2   3      AYMD       MAG       MADANG       MADANG  PAPUA NEW GUINEA   
3   4      AYMH       HGU  MOUNT HAGEN  MOUNT HAGEN  PAPUA NEW GUINEA   
4   5      AYNZ       LAE       NADZAB       NADZAB  PAPUA NEW GUINEA   

   lat_deg  lat_min  lat_sec lat_dir  lon_deg  lon_min  lon_sec lon_dir  \
0        6        4       54       S      145       23       30       E   
1        0        0        0       U        0        0        0       U   
2        5       12       25       S      145       47       19       E   
3        5       49       34       S      144       17       46       E   
4        6       34       11       S      146       43       34       E   

   altitude  lat_decimal  lon_decimal  
0      1610  

In [4]:
filtered_df = df[
    (df['lat_decimal'] != 0) & 
    (df['lon_decimal'] != 0) & 
    (df['icao_code'].notnull())
]

filtered_df[[
    'icao_code', 'iata_code', 'name', 'city', 'country',
    'lat_decimal', 'lon_decimal', 'altitude'
]].to_csv("data/airports_cleaned.csv", index=False)


NameError: name 'df' is not defined

In [4]:
import pandas as pd

aircraft_engine_combinations_df = pd.read_csv("data/aircraft_engine_combinations.csv", sep=';')
aircraft_engine_combinations_df.head(10)

,#,IDavion,JP,Production Aircraft,Manufacturer,TYPE,Version,Commercial Name,Modif. Number. air,Modif. descr. air,...,Lateral/Full Power(EPNdB),Limit Lateral/Full Power(EPNdB),Margin Lateral/Full Power(EPNdB),Approach(EPNdB),Limit APP(EPNdB),Margin APP(EPNdB),Interpolation FO,Interpolation Lateral/Full Power,Interpolation APP,Remarks
0,1,AIRBUS_15281,J,Out of Production,AIRBUS,A300,B4-605R,NaN,13219,Recertification to Chapter 4,...,97.5,99.9,2.4,99.6,103.3,3.7,0,0,0,"Weight variant n° 1, Dossier n° 00.X.009.10358..."
1,2,AIRBUS_15282,J,Out of Production,AIRBUS,A300,B4-605R,NaN,13219,Recertification to Chapter 4,...,97.5,99.9,2.4,99.6,103.3,3.7,0,0,0,"Weight variant n° 2, Dossier n° 00.X.009.10358..."
2,3,AIRBUS_15283,J,Out of Production,AIRBUS,A300,B4-605R,NaN,13219,Recertification to Chapter 4,...,97.5,99.9,2.4,99.6,103.3,3.7,0,0,0,"Weight variant n° 0, Dossier n° 00.X.009.10358..."
3,4,AIRBUS_15284,J,Out of Production,AIRBUS,A300,B4-605R,NaN,13219,Recertification to Chapter 4,...,97.5,99.8,2.3,99.6,103.3,3.7,0,0,0,"Weight variant n° 3, Dossier n° 00.X.009.10358..."
4,5,AIRBUS_15285,J,Out of Production,AIRBUS,A300,B4-605R,NaN,13219,Recertification to Chapter 4,...,97.9,99.4,1.5,99.6,102.9,3.3,0,0,0,"Weight variant n° 8, Dossier n° 00.X.009.10358..."
5,6,AIRBUS_15298,J,Out of Production,AIRBUS,A300,B4-620,NaN,NaN,NaN,...,98.8,99.7,0.9,100.8,103.2,2.4,0,0,0,"Weight variant n° 0, Dossier n° 00.X.009.10316..."
6,7,AIRBUS_15299,J,Out of Production,AIRBUS,A300,B4-622,NaN,13219,Recertification to Chapter 4,...,96.5,99.7,3.2,101.9,103.2,1.3,0,0,0,"Weight variant n° 0, Dossier n° 00.X.009.10360..."
7,8,AIRBUS_15300,J,Out of Production,AIRBUS,A300,B4-622,NaN,13219,Recertification to Chapter 4,...,96.8,99.5,2.7,101.9,103.0,1.1,0,0,0,"Weight variant n° 8, Dossier n° 00.X.009.10360..."
8,9,AIRBUS_15301,J,Out of Production,AIRBUS,A300,B4-622,NaN,13219,Recertification to Chapter 4,...,96.4,99.7,3.3,100.7,103.2,2.5,0,0,0,"Weight variant n° 0, Dossier n° 00.X.009.10359..."
9,10,AIRBUS_15302,J,Out of Production,AIRBUS,A300,B4-622,NaN,13219,Recertification to Chapter 4,...,96.7,99.5,2.8,100.7,103.0,2.3,0,0,0,"Weight variant n° 8, Dossier n° 00.X.009.10359..."


In [ ]:
# Example: filter and sort for A320-214 + CFM56-5B4/2P
aircraft_engine_combinations_df = aircraft_engine_combinations_df[~aircraft_engine_combinations_df["Engine"].str.contains(",", na=False)]

df_filtered = aircraft_engine_combinations_df[
    (aircraft_engine_combinations_df['TYPE'].str.contains("A320")) & 
    (aircraft_engine_combinations_df['Engine'].str.contains("CFM56-5B4/2P"))
]

# Sort by certification date or margin
df_filtered_sorted = df_filtered.sort_values("Certif Date", ascending=False)

In [1]:
from emissions_calculator import calc_noise_emissions

calc_noise_emissions()

In [1]:
from geo.route_mapper import map_flight_path
from geo.flight_map_plotter import plot_pollutant_emissions_map, plot_noise_emissions_map

start_location = map_flight_path()
plot_pollutant_emissions_map(start_location)
plot_noise_emissions_map(start_location)